In [1]:
!pip install scanpy scipy umap-learn leidenalg
!pip install hmmlearn
!pip install scikit-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 3.5 MB/s eta 0:00:00


Setup

In [3]:
import scanpy as sc
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import scipy.sparse
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_recall_fscore_support
from hmmlearn import hmm
from scipy import stats
from scipy.ndimage import uniform_filter1d
from scipy.ndimage import gaussian_filter1d
from scipy import stats
from scipy.signal import medfilt, savgol_filter
import re
import itertools

Fetch data and integrate into one

In [9]:
import scanpy as sc
adata1 = sc.read_10x_h5("/content/GSM7779484_SLX-21198.SITTH1.filtered_feature_bc_matrix.h5")
adata1.var_names_make_unique()
adata1.obs["sample"] = "SITTH1"

adata2 = sc.read_10x_h5("/content/GSM7779488_SLX-22979.SITTB3.filtered_feature_bc_matrix.h5")
adata2.var_names_make_unique()
adata2.obs["sample"] = "SITT3"

adata3 = sc.read_10x_h5("/content/GSM7779489_SLX-22979.SITTF2.filtered_feature_bc_matrix.h5")
adata3.var_names_make_unique()
adata3.obs["sample"] = "SITT2"

# merge
adata_merged = adata1.concatenate(adata2, adata3, batch_key="batch", batch_categories=["SITTH1", "SITT3", "SITT2"])


print(adata_merged)
print(adata_merged.obs["batch"].value_counts())
print(adata_merged.obs["sample"].value_counts())


import scanpy as sc
adata = sc.read_h5ad("PSC_CNA_dataset3_merged.h5ad")




/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_name

AnnData object with n_obs × n_vars = 22196 × 60624
    obs: 'sample', 'batch'
    var: 'gene_ids', 'feature_types', 'genome'
batch
SITTH1    7879
SITT3     7317
SITT2     7000
Name: count, dtype: int64
sample
SITTH1    7879
SITT3     7317
SITT2     7000
Name: count, dtype: int64


In [10]:
# normalization + log1p
if adata.X.max() > 100:
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)

# PCA
sc.pp.pca(adata)

# build neigbors
sc.pp.neighbors(adata)

# visualization（UMAP）
sc.tl.umap(adata)

# Leiden cluster
sc.tl.leiden(adata, key_added='leiden_clusters')

<ipython-input-10-2090faa52d8a>:16: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(adata, key_added='leiden_clusters')


Data Annotation

In [11]:
!pip install mygene

from mygene import MyGeneInfo
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.6 MB/s eta 0:00:00


In [12]:
mg = MyGeneInfo()

# get all gene names（default adata.var_names）
genes = adata.var_names.tolist()

# search for chromosome
query = mg.querymany(genes, scopes='symbol', fields='chromosome', species='human')


INFO:biothings.client:querying 1-1000 ...
INFO:biothings.client:querying 1001-2000 ...
INFO:biothings.client:querying 2001-3000 ...
INFO:biothings.client:querying 3001-4000 ...
INFO:biothings.client:querying 4001-5000 ...
INFO:biothings.client:querying 5001-6000 ...
INFO:biothings.client:querying 6001-7000 ...
INFO:biothings.client:querying 7001-8000 ...
INFO:biothings.client:querying 8001-9000 ...
INFO:biothings.client:querying 9001-10000 ...
INFO:biothings.client:querying 10001-11000 ...
INFO:biothings.client:querying 11001-12000 ...
INFO:biothings.client:querying 12001-13000 ...
INFO:biothings.client:querying 13001-14000 ...
INFO:biothings.client:querying 14001-15000 ...
INFO:biothings.client:querying 15001-16000 ...
INFO:biothings.client:querying 16001-17000 ...
INFO:biothings.client:querying 17001-18000 ...
INFO:biothings.client:querying 18001-19000 ...
INFO:biothings.client:querying 19001-20000 ...
INFO:biothings.client:querying 20001-21000 ...
INFO:biothings.client:querying 2100

In [13]:
from mygene import MyGeneInfo
mg = MyGeneInfo()

# insert adata.var_names as query
query = mg.querymany(
    adata.var_names.tolist(),
    scopes='symbol',
    fields='symbol,name,chromosome,genomic_pos',
    species='human'
)

# transform DataFrame and set index
df = pd.DataFrame(query).set_index('query')

# print out and check 'chromosome'、'genomic_pos'
print(df.columns)
print(df.head())

INFO:biothings.client:querying 1-1000 ...
INFO:biothings.client:querying 1001-2000 ...
INFO:biothings.client:querying 2001-3000 ...
INFO:biothings.client:querying 3001-4000 ...
INFO:biothings.client:querying 4001-5000 ...
INFO:biothings.client:querying 5001-6000 ...
INFO:biothings.client:querying 6001-7000 ...
INFO:biothings.client:querying 7001-8000 ...
INFO:biothings.client:querying 8001-9000 ...
INFO:biothings.client:querying 9001-10000 ...
INFO:biothings.client:querying 10001-11000 ...
INFO:biothings.client:querying 11001-12000 ...
INFO:biothings.client:querying 12001-13000 ...
INFO:biothings.client:querying 13001-14000 ...
INFO:biothings.client:querying 14001-15000 ...
INFO:biothings.client:querying 15001-16000 ...
INFO:biothings.client:querying 16001-17000 ...
INFO:biothings.client:querying 17001-18000 ...
INFO:biothings.client:querying 18001-19000 ...
INFO:biothings.client:querying 19001-20000 ...
INFO:biothings.client:querying 20001-21000 ...
INFO:biothings.client:querying 2100

Index(['_id', '_score', 'genomic_pos', 'name', 'symbol', 'notfound'], dtype='object')
                       _id     _score  \
query                                   
DDX11L1          100287102   8.425334   
DDX11L1    ENSG00000223972   8.425334   
WASH7P              653635   8.592555   
WASH7P     ENSG00000227232   8.592555   
MIR6859-1        102466751  27.432451   

                                                 genomic_pos  \
query                                                          
DDX11L1    {'chr': '1', 'end': 24894, 'ensemblgene': 'ENS...   
DDX11L1    {'chr': '1', 'end': 13670, 'ensemblgene': 'ENS...   
WASH7P     {'chr': '1', 'end': 30744, 'ensemblgene': 'ENS...   
WASH7P     {'chr': '1', 'end': 24886, 'ensemblgene': 'ENS...   
MIR6859-1  {'chr': '1', 'end': 17436, 'ensemblgene': 'ENS...   

                                                 name     symbol notfound  
query                                                                      
DDX11L1    DEAD/H-box hel

In [14]:
# process genomic_pos
def unpack_genomic_pos(row):
    pos = row.get('genomic_pos')
    if isinstance(pos, dict):
        return pd.Series({
            'chromosome': pos.get('chr'),
            'start': pos.get('start'),
            'end': pos.get('end')
        })
    else:
        return pd.Series({'chromosome': None, 'start': None, 'end': None})

# apply to dataframe
df_pos = df.apply(unpack_genomic_pos, axis=1)

# avoid duplicate index
df_pos_dedup = df_pos.loc[~df_pos.index.duplicated(keep='first')]

# align index
adata.var['chromosome'] = df_pos_dedup['chromosome'].reindex(adata.var_names)
adata.var['start'] = df_pos_dedup['start'].reindex(adata.var_names)
adata.var['end'] = df_pos_dedup['end'].reindex(adata.var_names)

Functions

In [15]:
def find_cluster_references(
    adata: sc.AnnData,
    cluster_key: str = 'leiden_clusters',
    layer: str | None = None,
    n_genes_subset: int = 3000,
    gmm_components: int = 2,
    random_state: int = 42,
    verbose: bool = False
) -> dict[str, list[str]]:
    """
    For each cluster in adata.obs[cluster_key], select a diploid reference
    via low‐variance cells. Returns { cluster: [cell, ...], ... }.
    """
    if cluster_key not in adata.obs:
        raise KeyError(f"{cluster_key!r} not in adata.obs")

    X = adata.layers[layer] if layer else adata.X
    if scipy.sparse.issparse(X):
        X = X.tocsr()

    nonmt = ~adata.var_names.str.upper().str.startswith('MT-')
    avail = np.where(nonmt)[0]
    if len(avail) >= n_genes_subset:
        rng = np.random.RandomState(random_state)
        subset_idx = rng.choice(avail, n_genes_subset, replace=False)
    else:
        subset_idx = avail if len(avail)>0 else np.arange(adata.n_vars)

    refs: dict[str, list[str]] = {}

    clusters = [str(x) for x in adata.obs[cluster_key].unique()]

    for cl in clusters:
        mask_cl = (adata.obs[cluster_key].astype(str).values == cl)
        idx_cl  = np.where(mask_cl)[0]
        if len(idx_cl) < 5:
            if verbose:
                print(f"Cluster {cl!r}: only {len(idx_cl)} cells → skipping")
            refs[cl] = []
            continue

        block = X[idx_cl[:, None], subset_idx]

        if scipy.sparse.issparse(block):
            block = block.toarray()

        vars_ = np.nanvar(block, axis=1, ddof=1).reshape(-1,1)

        gmm = GaussianMixture(n_components=gmm_components,
                              random_state=random_state)
        gmm.fit(vars_)
        lbl = gmm.predict(vars_)
        means = gmm.means_.flatten()
        dip = int(np.argmin(means))

        dip_idx   = idx_cl[lbl == dip]
        dip_cells = adata.obs_names[dip_idx].tolist()
        refs[cl]   = dip_cells

        if verbose:
            print(f"Cluster {cl!r}: {len(dip_cells)}/{len(idx_cl)} refs "
                  f"  (GMM means var={means.round(4)})")

    return refs

In [16]:
def detect_cnas_per_cluster(
    adata,
    cluster_key: str,
    cluster_refs: dict[str, list[str]],
    window_size: int = 100,
    min_genes_per_window: int = 10,
    smoothing_sigma: float = 1.0,
    lfc_threshold: float = 0.1,
    fraction_threshold: float = 0.2,
    layer: str | None = None
) -> tuple[pd.DataFrame, list[dict], pd.DataFrame]:
    """
    For each cluster in adata.obs[cluster_key], uses cluster_refs to compute
    smoothed per-window log2 fold-change and calls gains/losses by requiring
    that at least `fraction_threshold` of windows on a chromosome exceed
    +lfc_threshold (gain) or -lfc_threshold (loss).

    Returns
    -------
    calls_chr : pd.DataFrame
        DataFrame of shape (n_cells, n_chromosomes) with values in {-1, 0, +1}.
    """
    X = adata.layers[layer] if layer else adata.X
    if scipy.sparse.issparse(X):
        X = X.toarray()

    var = adata.var.copy()
    var['chrom'] = var['chromosome'].astype(str)
    var['start'] = pd.to_numeric(var['start'], errors='coerce')
    var = var.dropna(subset=['chrom','start'])
    def chr_key(c):
        c = str(c)
        return (int(c), '') if c.isdigit() else (np.inf, c)
    var['_idx'] = np.arange(var.shape[0])
    var = var.sort_values(['chrom','start'], key=lambda col: col.map(chr_key))
    sorted_idx = var['_idx'].values.astype(int)
    gene_chroms = var['chrom'].values

    Xs = X[:, sorted_idx]
    n_cells, n_win = Xs.shape

    windows = []
    window_expr = []
    for chrom in pd.unique(gene_chroms):
        idx_chr = np.where(gene_chroms == chrom)[0]
        step = max(1, window_size // 2)
        for i in range(0, len(idx_chr), step):
            block = idx_chr[i:i+window_size]
            if len(block) < min_genes_per_window:
                continue
            windows.append({'chrom': chrom, 'genes': block})
            window_expr.append(Xs[:, block].mean(axis=1))
    W = np.stack(window_expr, axis=1)

    chroms = sorted({w['chrom'] for w in windows}, key=chr_key)
    calls_chr = pd.DataFrame(0, index=adata.obs_names, columns=chroms, dtype=int)

    for cl in adata.obs[cluster_key].astype(str).unique():
        refs = cluster_refs.get(cl, [])
        if len(refs) < min_genes_per_window:
            continue
        mask_cl = (adata.obs[cluster_key].astype(str) == cl).values
        idx_cl = np.where(mask_cl)[0]
        mask_ref = adata.obs_names.isin(refs)
        idx_ref = np.where(mask_ref)[0]
        if len(idx_ref) == 0:
            continue
        ref_med = np.median(W[idx_ref, :], axis=0)
        ref_med = np.clip(ref_med, 1e-6, None)
        L = np.log2((W[idx_cl, :] + 1e-6) / ref_med[np.newaxis, :])
        if smoothing_sigma > 0:
            for chrom in chroms:
                win_idxs = [i for i,w in enumerate(windows) if w['chrom']==chrom]
                if len(win_idxs) > 1:
                    L[:, win_idxs] = gaussian_filter1d(
                        L[:, win_idxs],
                        sigma=smoothing_sigma,
                        axis=1,
                        mode='reflect'
                    )
        for chrom in chroms:
            win_idxs = [i for i,w in enumerate(windows) if w['chrom']==chrom]
            sub = L[:, win_idxs]
            frac_gain = (sub > lfc_threshold).sum(axis=1) / len(win_idxs)
            frac_loss = (sub < -lfc_threshold).sum(axis=1) / len(win_idxs)
            cells = adata.obs_names[idx_cl]
            calls_chr.loc[cells[frac_gain >= fraction_threshold], chrom] = +1
            calls_chr.loc[cells[frac_loss >= fraction_threshold], chrom] = -1

    return calls_chr, windows, var

In [17]:
def extract_cna_segments_fast(
    calls_chr: pd.DataFrame,
    windows: list[dict],
    var: pd.DataFrame
) -> pd.DataFrame:
    """
    Vectorized version of extract_cna_segments (faster).
    """
    segment_list = []

    for w in windows:
        chrom = w['chrom']
        gene_inds = w['genes']
        gene_names = var.index[gene_inds]

        start = var.loc[gene_names, 'start'].min()
        end   = var.loc[gene_names, 'end'].max()

        if chrom not in calls_chr.columns:
            continue

        subset = calls_chr[chrom]
        mask = subset != 0

        seg = pd.DataFrame({
            'cell': subset.index[mask],
            'chromosome': chrom,
            'start': start,
            'end': end,
            'genotype': subset[mask].map({1: 'gain', -1: 'loss'})
        })

        segment_list.append(seg)

    return pd.concat(segment_list, ignore_index=True)


Find reference

In [18]:
cluster_refs = find_cluster_references(
    adata,
    cluster_key='leiden_clusters',
    n_genes_subset=3000,
    gmm_components=2,
    random_state=42,
    verbose=True
)


Cluster '1': 1371/2875 refs   (GMM means var=[0.0689 0.0819])
Cluster '4': 1132/1690 refs   (GMM means var=[0.0676 0.0774])
Cluster '9': 466/931 refs   (GMM means var=[0.0809 0.068 ])
Cluster '12': 578/687 refs   (GMM means var=[0.0766 0.0677])
Cluster '5': 598/1396 refs   (GMM means var=[0.0595 0.075 ])
Cluster '16': 230/296 refs   (GMM means var=[0.0664 0.0797])
Cluster '13': 354/672 refs   (GMM means var=[0.0734 0.0871])
Cluster '6': 911/1372 refs   (GMM means var=[0.0882 0.0791])
Cluster '8': 809/1342 refs   (GMM means var=[0.0692 0.0791])
Cluster '10': 445/876 refs   (GMM means var=[0.0782 0.0886])
Cluster '7': 914/1346 refs   (GMM means var=[0.0821 0.0917])
Cluster '15': 263/393 refs   (GMM means var=[0.0869 0.0791])
Cluster '0': 1759/3191 refs   (GMM means var=[0.0809 0.0899])
Cluster '2': 1008/1838 refs   (GMM means var=[0.082  0.0753])
Cluster '3': 1015/1771 refs   (GMM means var=[0.0859 0.0764])
Cluster '14': 354/573 refs   (GMM means var=[0.0744 0.0843])
Cluster '11': 404/73

Detect CNA

In [19]:
calls_chr, windows, var_sorted = detect_cnas_per_cluster(
    adata,
    cluster_key='leiden_clusters',
    cluster_refs=cluster_refs,
    window_size=100,
    min_genes_per_window=10,
    smoothing_sigma=1.0,
    lfc_threshold=0.05,
    fraction_threshold=0.2,
    layer=None
)

Segment

In [20]:
segments_df = extract_cna_segments_fast(
    calls_chr=calls_chr,
    windows=windows,
    var=adata.var
)


In [21]:
cna_segments = extract_cna_segments_fast(calls_chr, windows, var_sorted)

In [22]:
cna_segments = (
    cna_segments
    .merge(
        adata.obs[['leiden_clusters']],
        left_on='cell',
        right_index=True
    )
)
cna_segments.head()

,cell,chromosome,start,end,genotype,leiden_clusters
0,AAACCCAAGCACTAAA-1-SITTH1,1,1001138.0,109600195.0,loss,1
1,AAACCCAAGCAGTAAT-1-SITTH1,1,1001138.0,109600195.0,loss,1
2,AAACCCAAGCGCCTAC-1-SITTH1,1,1001138.0,109600195.0,loss,4
3,AAACCCAAGGGTTTCT-1-SITTH1,1,1001138.0,109600195.0,loss,1
4,AAACCCAAGTGATAAC-1-SITTH1,1,1001138.0,109600195.0,gain,9


Summarize

In [23]:
import pandas as pd

def summarize_chr_cna(segments_df: pd.DataFrame) -> pd.DataFrame:
    summary = (
        segments_df.groupby(['chromosome', 'genotype'])
        .size()
        .unstack(fill_value=0)
        .rename_axis(None, axis=1)
        .reset_index()
    )

    # make up all the columns
    if 'gain' not in summary.columns:
        summary['gain'] = 0
    if 'loss' not in summary.columns:
        summary['loss'] = 0

    return summary.sort_values('chromosome')

In [24]:
summary_df = summarize_chr_cna(segments_df)
display(summary_df)

,chromosome,gain,loss
0,1,122264,1387064
1,10,73687,614389
2,11,87494,822542
3,12,71298,683366
4,13,74361,391755
5,14,89622,487448
6,15,80753,429732
7,16,58275,496625
8,17,90496,619776
9,18,59400,206892
